# Python Web API Tutorial
> A discussion on Web APIs.  This will include Python at the center of using 3rd party APIs (like RapidAPI), creating APIs (like Jokes), and creating API that work with persistent databases (like SQLite).

- toc: true
- comments: true
- categories: [pbl]
- permalink: /tutorial/webapi

### Model for API
> We will begin our journey into APIs by creating and thinking about data.  We have learned about Python Lists and dictionaries.  In this data example, we are going to make "the best computer jokes ever" and serve them to the Internet.  The ultimate objective is to allow our viewers to give feedback on our jokes.

- This begins by coming up with some jokes and defining a data "model" to keep and manage the jokes.
    - jokes_data   contains a list of dictionary records containing joke and reactions: haha or boohoo
    - joke_list   contains collection of jokes we will put into jokes_data
- Next comes some functions to interact with our jokes
    - def initJokes(): initializes jokes_data
    - def getJokes(): returns the complete list of jokes
    - def getJoke():  returns a single joke from our list

In [1]:
import random

jokes_data = []
joke_list = [
    "If you give someone a program... you will frustrate them for a day; if you teach them how to program... you will "
    "frustrate them for a lifetime.",
    "Q: Why did I divide sin by tan? A: Just cos.",
    "UNIX is basically a simple operating system... but you have to be a genius to understand the simplicity.",
    "Enter any 11-digit prime number to continue.",
    "If at first you don't succeed; call it version 1.0.",
    "Java programmers are some of the most materialistic people I know, very object-oriented",
    "The oldest computer can be traced back to Adam and Eve. It was an apple but with extremely limited memory. Just "
    "1 byte. And then everything crashed.",
    "Q: Why did Wi-Fi and the computer get married? A: Because they had a connection",
    "Bill Gates teaches a kindergarten class to count to ten. 1, 2, 3, 3.1, 95, 98, ME, 2000, XP, Vista, 7, 8, 10.",
    "Q: What’s a aliens favorite computer key? A: the space bar!",
    "There are 10 types of people in the world: those who understand binary, and those who don’t.",
    "If it wasn't for C, we’d all be programming in BASI and OBOL.",
    "Computers make very fast, very accurate mistakes.",
    "Q: Why is it that programmers always confuse Halloween with Christmas? A: Because 31 OCT = 25 DEC.",
    "Q: How many programmers does it take to change a light bulb? A: None. It’s a hardware problem.",
    "The programmer got stuck in the shower because the instructions on the shampoo bottle said: Lather, Rinse, Repeat.",
    "Q: What is the biggest lie in the entire universe? A: I have read and agree to the Terms and Conditions.",
    'An SQL statement walks into a bar and sees two tables. It approaches, and asks may I join you?'
]

# Initialize jokes
def initJokes():
    # setup jokes into a dictionary with id, joke, haha, boohoo
    item_id = 0
    for item in joke_list:
        jokes_data.append({"id": item_id, "joke": item, "haha": 0, "boohoo": 0})
        item_id += 1
    # prime some haha responses
    for i in range(200):
        id = getRandomJoke()['id']
        addJokeHaHa(id)
    # prime some haha responses
    for i in range(50):
        id = getRandomJoke()['id']
        addJokeBooHoo(id)
        
# Return all jokes from jokes_data
def getJokes():
    return(jokes_data)

# Joke getter
def getJoke(id):
    return(jokes_data[id])

# Return random joke from jokes_data
def getRandomJoke():
    return(random.choice(jokes_data))

# Liked joke
def favoriteJoke():
    best = 0
    bestID = -1
    for joke in getJokes():
        if joke['haha'] > best:
            best = joke['haha']
            bestID = joke['id']
    return jokes_data[bestID]
    
# Jeered joke
def jeeredJoke():
    worst = 0
    worstID = -1
    for joke in getJokes():
        if joke['boohoo'] > worst:
            worst = joke['boohoo']
            worstID = joke['id']
    return jokes_data[worstID]

# Add to haha for requested id
def addJokeHaHa(id):
    jokes_data[id]['haha'] = jokes_data[id]['haha'] + 1
    return jokes_data[id]['haha']

# Add to boohoo for requested id
def addJokeBooHoo(id):
    jokes_data[id]['boohoo'] = jokes_data[id]['boohoo'] + 1
    return jokes_data[id]['boohoo']

# Pretty Print joke
def printJoke(joke):
    print(joke['id'], joke['joke'], "\n", "haha:", joke['haha'], "\n", "boohoo:", joke['boohoo'], "\n")

# Number of jokes
def countJokes():
    return len(jokes_data)

# Test Joke Model
if __name__ == "__main__": 
    initJokes()  # initialize jokes
    
    # Most likes and most jeered
    best = favoriteJoke()
    print("Most liked", best['haha'])
    printJoke(best)
    worst = jeeredJoke()
    print("Most jeered", worst['boohoo'])
    printJoke(worst)
    
    # Random joke
    print("Random joke")
    printJoke(getRandomJoke())
    
    # Count of Jokes
    print("Jokes Count: " + str(countJokes()))

Most liked 17
10 There are 10 types of people in the world: those who understand binary, and those who don’t. 
 haha: 17 
 boohoo: 1 

Most jeered 9
12 Computers make very fast, very accurate mistakes. 
 haha: 15 
 boohoo: 9 

Random joke
16 Q: What is the biggest lie in the entire universe? A: I have read and agree to the Terms and Conditions. 
 haha: 12 
 boohoo: 1 

Jokes Count: 18


### Backend Interface for API (Control)


In [4]:
from flask import Blueprint, jsonify
from flask_restful import Api, Resource
import requests
import random

app_api = Blueprint('api', __name__,
                   url_prefix='/api/jokes')

# API generator https://flask-restful.readthedocs.io/en/latest/api.html#id1
api = Api(app_api)

class JokesAPI:
    # class for create/post
    class _Create(Resource):
        def post(self, joke):
            pass
            
    # class for read/get
    class _Read(Resource):
        def get(self):
            return jsonify(getJokes())

    # class for delete
    class _ReadID(Resource):
        def get(self, id):
            return jsonify(getJoke(id))

    # class for read/get
    class _ReadRandom(Resource):
        def get(self):
            return jsonify(getRandomJoke())

    # class for update/put
    class _UpdateLike(Resource):
        def put(self, id):
            addJokeHaHa(id)
            return jsonify(getJoke(id))

    class _UpdateJeer(Resource):
        def put(self, id):
            addJokeBooHoo(id)
            return jsonify(getJoke(id))

    # building RESTapi resource
    # building RESTapi resource
    api.add_resource(_Create, '/create/<string:joke>')
    api.add_resource(_Read, '/')
    api.add_resource(_ReadID, '/<int:id>')
    api.add_resource(_ReadRandom, '/random')
    api.add_resource(_UpdateLike, '/like/<int:id>/')
    api.add_resource(_UpdateJeer, '/jeer/<int:id>/')
    
if __name__ == "__main__": 
    # server = "http://127.0.0.1:5000/" # run local
    server = 'https://flask.nighthawkcodingsociety.com/' # run from web
    url = server + "api/jokes/"
    responses = []  # responses list
    
    # responses.append(requests.get(url)) # read all, verbose so commented out
    responses.append(requests.get(url+"random")) # read a random joke
    
    # update likes/dislikes test sequence
    num = str(random.randint(0, countJokes()-1)) # test a random record
    responses.append(
        requests.get(url+num)  # read joke by id
        ) 
    responses.append(
        requests.put(url+"like/"+num) # add to like count
        ) 
    responses.append(
        requests.put(url+"jeer/"+num) # add to jeer count
        ) 

    # cycle through responses
    for response in responses:
        print(response)
        try:
            print(response.json())
        except:
            print("unknown error")

<Response [200]>
{'boohoo': 9, 'haha': 10, 'id': 5, 'joke': 'Java programmers are some of the most materialistic people I know, very object-oriented'}
<Response [200]>
{'boohoo': 2, 'haha': 9, 'id': 10, 'joke': 'There are 10 types of people in the world: those who understand binary, and those who don’t.'}
<Response [200]>
{'boohoo': 2, 'haha': 10, 'id': 10, 'joke': 'There are 10 types of people in the world: those who understand binary, and those who don’t.'}
<Response [200]>
{'boohoo': 3, 'haha': 10, 'id': 10, 'joke': 'There are 10 types of people in the world: those who understand binary, and those who don’t.'}
